In [1]:
from grandma_stock_valuation import FileLogger,loadPacakgeData
from grandma_stock_valuation import batchValuation, caliValueWithGrowth, addCashPortfolio
from grandma_stock_valuation import getCorrelationWeight, allocatePortfolio

logger = FileLogger()
logPrint = logger.logPandas


### Load Data

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

### Grandma Valuation

In [3]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, draw_figure=False, verbose=0, printfunc=logPrint)
df_valuation_metrics['over_value_score'] = caliValueWithGrowth(
    df_valuation_metrics['over_value_range'],
    df_valuation_metrics['annualized_return'],
    method='years', # 'subtract'
    growth_scale=0.2
    )
df_valuation_metrics['name'] = df_valuation_metrics['ticker'].apply(lambda s: d_instrument[s])

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_score,name
0,VPL,0.890218,10.00274,0.067348,74.18,77.196246,-0.039072,-0.026315,Developed Asia-Pacific
1,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.048463,SP500
2,EEMA,0.847897,10.00274,0.065518,77.550003,79.470586,-0.024167,-0.015834,Emerging Asia
3,IEV,0.780975,10.00274,0.052644,50.130001,49.242054,0.018032,0.034253,Europe


### Grandma Portfolio Allocation

In [22]:
instruments_select = ['IVV', 'EEMA', 'VPL', 'IEV']
df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()

df_portfolio = addCashPortfolio(df_portfolio, value_col='over_value_score')

d_instrument_prices = {ticker:data for ticker, data in d_instrument_data.items() if ticker in instruments_select}
d_cor_weights = getCorrelationWeight(d_instrument_prices, with_cash=True, cash_name='cash', verbose=0)

df_portfolio['cor_weight'] = df_portfolio['ticker'].apply(lambda t: d_cor_weights[t])

weights = df_portfolio['cor_weight']
df_portfolio['portfolio'] = allocatePortfolio(df_portfolio['over_value_score'], scale=20, transformation='sigmoid', weights=weights)

df_portfolio

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_score,name,cor_weight,portfolio
0,VPL,0.890218,10.00274,0.067348,74.18,77.196246,-0.039072,-0.052542,Developed Asia-Pacific,0.133296,0.190646
1,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.038230,SP500,0.220193,0.135015
2,EEMA,0.847897,10.00274,0.065518,77.550003,79.470586,-0.024167,-0.037271,Emerging Asia,0.198516,0.259878
3,IEV,0.780975,10.00274,0.052644,50.130001,49.242054,0.018032,0.007503,Europe,0.247995,0.221429
4,cash,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,0.200000,0.193032
